In [4]:
from pathlib import Path
from typing import Annotated

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import yaml
from pytask import Product

from caregiving.config import ROOT, SRC, BLD, JET_COLOR_MAP
from caregiving.model.shared import FEMALE, MALE

from caregiving.utils import table

In [6]:
path_to_parent_child_main = BLD / "data" / "share_parent_child_data.csv"

In [7]:
main = pd.read_csv(path_to_parent_child_main)

In [10]:
list(main)

['mergeid',
 'int_year',
 'int_month',
 'gender',
 'mobirth',
 'yrbirth',
 'age_int',
 'hhsize',
 'mergeidp',
 'coupleid',
 'ph003_',
 'ch005_1',
 'ch005_2',
 'ch005_3',
 'ch005_4',
 'ch005_5',
 'ch005_6',
 'ch005_7',
 'ch005_8',
 'ch005_9',
 'ch005_10',
 'ch005_11',
 'ch005_12',
 'ch005_13',
 'ch005_14',
 'ch005_15',
 'ch005_16',
 'ch005_17',
 'ch005_18',
 'ch005_19',
 'ch005_20',
 'ch006_1',
 'ch006_2',
 'ch006_3',
 'ch006_4',
 'ch006_5',
 'ch006_6',
 'ch006_7',
 'ch006_8',
 'ch006_9',
 'ch006_10',
 'ch006_11',
 'ch006_12',
 'ch006_13',
 'ch006_14',
 'ch006_15',
 'ch006_16',
 'ch006_17',
 'ch006_18',
 'ch006_19',
 'ch006_20',
 'ch007_1',
 'ch007_2',
 'ch007_3',
 'ch007_4',
 'ch007_5',
 'ch007_6',
 'ch007_7',
 'ch007_8',
 'ch007_9',
 'ch007_10',
 'ch007_11',
 'ch007_12',
 'ch007_13',
 'ch007_14',
 'ch007_15',
 'ch007_16',
 'ch007_17',
 'ch007_18',
 'ch007_19',
 'ch007_20',
 'ch012_1',
 'ch012_2',
 'ch012_3',
 'ch012_4',
 'ch012_5',
 'ch012_6',
 'ch012_7',
 'ch012_8',
 'ch012_9',
 'ch0

In [11]:
table(main["adl_cat"])

adl_cat
0.0    27613
1.0      758
2.0      431
3.0      283
Name: Count, dtype: int64

In [12]:
main["any_adl"] = (main["adl_cat"] >= 1).copy()

In [14]:
table(main["any_adl"])

any_adl
False    28192
True      1472
Name: Count, dtype: int64

In [18]:
main.loc[main["adl_cat"] >= 1, "informal_care_general"].mean()

np.float64(0.5137254901960784)

In [19]:
table(main["informal_care_child"])

informal_care_child
0.0     363
1.0    2095
Name: Count, dtype: int64

In [20]:
table(main["informal_care_general"])

informal_care_general
0.0    16589
1.0     5655
Name: Count, dtype: int64

In [26]:
bins = [50, 60, 70, 80, 90, 100]
labels = ["50-60", "60-70", "70-80", "80-90", "90-100"]
main["age_group"] = pd.cut(main["age"], bins=bins, labels=labels, right=False)
mean_care = main.loc[main["adl_cat"] >= 1].groupby("age_group")["informal_care_general"].mean()

/tmp/ipykernel_82261/3047637161.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_care = main.loc[main['adl_cat'] >= 1].groupby('age_group')['informal_care_general'].mean()


In [27]:
mean_care

age_group
50-60     0.471429
60-70     0.439024
70-80     0.482480
80-90     0.572139
90-100    0.610619
Name: informal_care_general, dtype: float64

In [28]:
# 1) Define the 10-year age bins and labels
bins   = [60, 70, 80, 90, 100]          # 60-69, 70-79, 80-89, 90-99
labels = ["60-69", "70-79", "80-89", "90-99"]

# 2) Keep only respondents with adl_cat >= 1
df = main.loc[main["adl_cat"] >= 1].copy()

# 3) Create an age-group column
df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=False)

# 4) Compute the mean by age group
mean_by_age = df.groupby("age_group")["informal_care_general"].mean()

print(mean_by_age)

age_group
60-69    0.439024
70-79    0.482480
80-89    0.572139
90-99    0.610619
Name: informal_care_general, dtype: float64


/tmp/ipykernel_82261/1315687632.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_by_age = df.groupby('age_group')['informal_care_general'].mean()


In [32]:
# 1) Filter to respondents with at least one ADL limitation
df = main.loc[main["adl_cat"] >= 1].copy()

# 2) Build 5-year age bins: 60-64, 65-69, …, 95-99
bins   = np.arange(60, 105, 10)                    # [60, 65, 70, …, 100]
labels = [f"{b}-{b+9}" for b in bins[:-1]]        # ['60-64', '65-69', …, '95-99']

df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=False)

# 3) Group by age bin
grp = df.groupby("age_group")

# 4) Share (proportion) receiving any informal care
share = grp["informal_care_general"].apply(lambda x: (x > 0).mean())

# 5) Number of observations
n_obs = grp.size()

# 6) Combine results
result = pd.concat([share, n_obs], axis=1)                # columns: 0 -> share, 1 -> n_obs
result.columns = ["share_receiving_care", "n_obs"]

print(result)

           share_receiving_care  n_obs
age_group                             
60-69                  0.373702    289
70-79                  0.417249    429
80-89                  0.497835    462
90-99                  0.543307    127


/tmp/ipykernel_82261/302128507.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp = df.groupby('age_group')


In [33]:
# 1) Keep only respondents with at least one ADL limitation
df = main.loc[main["adl_cat"] >= 1].copy()

# 2) Define 10-year age bins: 60-69, 70-79, 80-89, 90-99
bins   = [60, 70, 80, 90, 100]                  # upper edge 100 covers 90-99
labels = [f"{b}-{b+9}" for b in bins[:-1]]      # ['60-69', '70-79', '80-89', '90-99']

df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=False)

# 3) Group by age bin
grp = df.groupby("age_group")

# 4) Share (proportion) receiving any informal care
share = grp["informal_care_general"].apply(lambda x: (x > 0).mean())

# 5) Number of observations
n_obs = grp.size()

# 6) Combine results
result = pd.concat([share, n_obs], axis=1)
result.columns = ["share_receiving_care", "n_obs"]

print(result)

           share_receiving_care  n_obs
age_group                             
60-69                  0.373702    289
70-79                  0.417249    429
80-89                  0.497835    462
90-99                  0.543307    127


/tmp/ipykernel_82261/2389487763.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp = df.groupby('age_group')


In [40]:
# 1) Filter to the target population
df_home = main.loc[
    (main["adl_cat"] >= 0)
    & ((main["home_care"] == 1) | (main["informal_care_general"] > 0))
].copy()

# 2) Define 10-year age bins: 60-69, 70-79, 80-89, 90-99
bins   = [60, 70, 80, 90, 100]
labels = [f"{b}-{b+9}" for b in bins[:-1]]

df_home["age_group"] = pd.cut(df_home["age"], bins=bins, labels=labels, right=False)

# 3) Group by age bin
grp = df_home.groupby("age_group")

# 4) Share of respondents receiving informal care (informal_care_general > 0)
share = grp["informal_care_general"].apply(lambda x: (x > 0).mean())

# 5) Number of observations per age bin
n_obs = grp.size()

# 6) Combine results
result = pd.concat([share, n_obs], axis=1)
result.columns = ["share_receiving_informal_care", "n_obs"]

print(result)

           share_receiving_informal_care  n_obs
age_group                                      
60-69                           0.875068   1825
70-79                           0.794624   1860
80-89                           0.753343   1346
90-99                           0.696262    214


/tmp/ipykernel_82261/1765735956.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp = df_home.groupby('age_group')
